In [1]:
from pathlib import Path
import sys
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import torch
import factorizer as ft
from monai import transforms

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

### Inference on ISLES

In [2]:
# load the weights into the model
checkpoint_root = Path("/Data/logs/isles2022-dwi&adc/fold0/image-segmentation-factorizer/51ij4wr5/checkpoints/epoch=999-step=100000.ckpt")
model = ft.SemanticSegmentation.load_from_checkpoint(checkpoint_root).to(device)
model.eval()

Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.


SemanticSegmentation(
  (net): SegmentationFactorizer(
    (stem): Conv3d(2, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (encoder): UNetEncoder(
      (blocks): ModuleList(
        (0): UNetEncoderBlock(
          (blocks): Sequential(
            (0): FactorizerBlock(
              (blocks): ModuleDict(
                (nmf): Residual(
                  (fn): Sequential(
                    (0): LayerNorm(
                      (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
                    )
                    (1): FactorizerSubblock(
                      (in_proj): Linear(
                        (flatten): Flatten(start_dim=2, end_dim=-1)
                        (linear): Conv1d(32, 32, kernel_size=(1,), stride=(1,), bias=False)
                      )
                      (tensorize): SWMatricize(
                        (shifted_windows): ModuleList(
                          (0-1): 2 x Matricize(
                           

In [ ]:
# load the datamodule
from monai.metrics import DiceMetric
from monai.transforms import AsDiscrete

# Rebuild dm for the full dataset using the JSON properties file
dm = ft.ISLESDataModule(
    data_properties="/Data/data/isles/dataset_without_flair.json",
    spacing=[2.0, 2.0, 2.0],
    spatial_size=[64, 64, 64],
    num_workers=24,
    cache_num=4,
    cache_rate=1.0,
    batch_size=1,
    seed=42,
)
dm.test_transform = transforms.Compose([
    ft.ReadImaged(['input', 'label'], allow_missing_keys=False),
    transforms.NormalizeIntensityd('input', nonzero=True, channel_wise=True),
    transforms.ToTensord(['input', 'label']),
])
dm.setup('test')

In [29]:
# Metric and postprocessing
dice_metric = DiceMetric(include_background=True, reduction="mean")
post_pred = AsDiscrete(threshold=0.5)
post_label = AsDiscrete(threshold=0.5)

model.eval()
dice_scores = []

with torch.no_grad():
    for batch in dm.test_dataloader():
        batch['input'] = batch['input'].to(device)
        batch['label'] = batch['label'].to(device)
        pred = model.inferer.get_postprocessed(batch, model)
        print(5)
        print(type(pred))
        
        dice_metric(
            y_pred=post_pred(pred['input']),
            y=post_label(batch['label']),
        )
        
dice_scores = dice_metric.aggregate(reduction="mean_batch").cpu().numpy()
dice_metric.reset()

print("Dataset Dice (mean):", dice_scores.mean())

ValueError: the data to aggregate must be PyTorch Tensor.